In [12]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [17]:
import numpy as np
import pandas as pd
import datetime as dt
from pprint import pprint

# Reflect Tables into SQLAlchemy ORM

In [25]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func , inspect

In [19]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [20]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [21]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [22]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [23]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [26]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [27]:
# Get a list of column names and types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [28]:
# Get a list of column names and types
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [29]:
# Find the most recent date in the data set.
max_date = session.query(func.max(measurement.date)).first()[0]
print(max_date)

2017-08-23


In [30]:
min_date = session.query(func.min(measurement.date)).first()[0]
print(min_date)

2010-01-01


In [46]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)
# Perform a query to retrieve the data and precipitation scores

one_year_perc = session.query(measurement.date, measurement.prcp).filter(measurement.date<"2017-08-23")\
.filter(measurement.date>"2016-08-23").order_by(measurement.date).all()
#pprint(one_year_perc)
# Save the query results as a Pandas DataFrame and set the index to the date column
prec_df = pd.DataFrame(one_year_perc, columns = ["Date", "Precipitation"])
prec_df.set_index("Date", inplace = True)

prec_df
# Sort the dataframe by date
sort_prec_df=prec_df.sort_values("Date",  ascending=True)
pprint(sort_prec_df)
# Use Pandas Plotting with Matplotlib to plot the data




Query Date:  2016-08-23
            Precipitation
Date                     
2016-08-24           0.08
2016-08-24           2.15
2016-08-24           2.28
2016-08-24            NaN
2016-08-24           1.22
...                   ...
2017-08-21            NaN
2017-08-21           0.56
2017-08-22           0.00
2017-08-22           0.00
2017-08-22           0.50

[2219 rows x 1 columns]


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()